# Tarea 2 - Minería de Datos
Este notebook tiene como objetivo comparar el desempeño de dos modelos de clasificación supervisada para predecir la variable `categoria_objetivo` a partir de un conjunto de productos.

Modelos evaluados:
- Árbol de Decisión (`DecisionTreeClassifier`)
- Perceptrón Multicapa (`MLPClassifier`)

Se utilizará el dataset `productos.csv` y se aplicará validación cruzada para evaluar el rendimiento de los modelos.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, f1_score

import warnings
warnings.filterwarnings("ignore")


## Carga del dataset

In [ ]:
df = pd.read_csv("productos.csv")
df.head()


## Análisis exploratorio de datos

In [ ]:
df.info()


In [ ]:
df.describe(include="all")


In [ ]:
df['categoria_objetivo'].value_counts().sort_index()


## Preprocesamiento de datos

In [ ]:
# Variables numéricas y categóricas
numericas = ['peso_kg', 'volumen_m3']
categoricas = ['tipo_certificacion']

X = df[numericas + categoricas]
y = df['categoria_objetivo']

# Pipeline de preprocesamiento
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numericas),
    ('cat', OneHotEncoder(drop='first'), categoricas)
])


## División en conjunto de entrenamiento y prueba

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)


## Entrenamiento de modelos y comparación

In [ ]:
dt_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

dt_params = {
    'classifier__criterion': ['gini', 'entropy'],
    'classifier__max_depth': [3, 5, 10, None]
}

grid_dt = GridSearchCV(dt_pipeline, dt_params, cv=5, scoring='f1_macro')
grid_dt.fit(X_train, y_train)

print("Mejores parámetros Árbol de Decisión:")
print(grid_dt.best_params_)


In [ ]:
mlp_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', MLPClassifier(max_iter=500, random_state=42))
])

mlp_params = {
    'classifier__hidden_layer_sizes': [(10,), (50,), (50, 30)],
    'classifier__learning_rate_init': [0.01, 0.001],
    'classifier__alpha': [0.0001, 0.001]
}

grid_mlp = GridSearchCV(mlp_pipeline, mlp_params, cv=5, scoring='f1_macro')
grid_mlp.fit(X_train, y_train)

print("Mejores parámetros MLP:")
print(grid_mlp.best_params_)


## Evaluación en conjunto de prueba

In [ ]:
models = {
    "Árbol de Decisión": grid_dt.best_estimator_,
    "MLP": grid_mlp.best_estimator_
}

for nombre, modelo in models.items():
    y_pred = modelo.predict(X_test)
    print(f"Modelo: {nombre}")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("F1 macro:", f1_score(y_test, y_pred, average='macro'))
    print("Reporte de clasificación:")
    print(classification_report(y_test, y_pred))
    print("-" * 60)


## Matriz de confusión

In [ ]:
for nombre, modelo in models.items():
    y_pred = modelo.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.title(f"Matriz de confusión - {nombre}")
    plt.show()


## Conclusiones

Se evaluaron dos modelos de clasificación supervisada. El Árbol de Decisión mostró una mayor interpretabilidad, mientras que el MLP presentó un mejor rendimiento en términos de F1 macro en varios casos. La elección del modelo final dependerá del balance entre rendimiento y capacidad de interpretación requerida para el problema.
